# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,279.52,91,97,14.00,TF,1739552964
1,1,harstad,68.7990,16.5293,268.38,77,44,4.47,NO,1739553199
2,2,puerto natales,-51.7236,-72.4875,287.40,44,40,9.26,CL,1739553200
3,3,adamstown,-25.0660,-130.1015,298.42,83,100,9.85,PN,1739553201
4,4,isafjordur,66.0755,-23.1240,271.59,84,99,3.74,IS,1739553202


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',                  # X-axis: Longitude
    'Lat',                  # Y-axis: Latitude
    geo=True,              # Enable geographic plotting
    size='Humidity',       # Point size varies with humidity
    scale=1,              # Scale factor for point sizes
    color='City',         # Color points by city
    tiles='OSM',          # Use OpenStreetMap tiles
    frame_width=700,      # Set frame width
    frame_height=500,     # Set frame height
    title="City Humidity Map" # Set title
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (city_data_df["Max Temp"] > 20) & (city_data_df["Wind Speed"] < 10)
narrowed_down_cities_df = city_data_df[criteria]

# Drop any rows with null values
narrowed_down_cities_df = narrowed_down_cities_df.dropna()

# Display sample data
print(narrowed_down_cities_df.head())

   City_ID            City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
1        1         harstad  68.7990   16.5293    268.38        77          44   
2        2  puerto natales -51.7236  -72.4875    287.40        44          40   
3        3       adamstown -25.0660 -130.1015    298.42        83         100   
4        4      isafjordur  66.0755  -23.1240    271.59        84          99   
5        5          vuktyl  63.8567   57.3094    249.28       100          50   

   Wind Speed Country        Date  
1        4.47      NO  1739553199  
2        9.26      CL  1739553200  
3        9.85      PN  1739553201  
4        3.74      IS  1739553202  
5        1.49      RU  1739553004  


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

                City Country      Lat       Lng  Humidity Hotel Name
0  port-aux-francais      TF -49.3500   70.2167        91           
1            harstad      NO  68.7990   16.5293        77           
2     puerto natales      CL -51.7236  -72.4875        44           
3          adamstown      PN -25.0660 -130.1015        83           
4         isafjordur      IS  66.0755  -23.1240        84           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 5000  # Set the radius to 5000 meters (5 km)
params = {
    "categories": "accommodation.hotel",
    "apiKey": "YOUR_GEOAPIFY_API_KEY",  # Replace with your Geoapify API key
    "limit": 1  # Limit to the nearest hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
port-aux-francais - nearest hotel: No hotel found
harstad - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
vuktyl - nearest hotel: No hotel found
resplendor - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
naracoorte - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
ulsteinvik - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
puerto santa - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
forster - nearest hotel: No hotel found
shchuchinsk - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
metu - nearest hotel: No hotel found
puerto a

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,91,No hotel found
1,harstad,NO,68.7990,16.5293,77,No hotel found
2,puerto natales,CL,-51.7236,-72.4875,44,No hotel found
3,adamstown,PN,-25.0660,-130.1015,83,No hotel found
4,isafjordur,IS,66.0755,-23.1240,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    hover_cols=['City', 'Hotel Name'],  # Show city and hotel name on hover
    size=8,                            # Fixed point size
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    title="Hotels in Cities with Ideal Weather"
)
 
# Display the hotel map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)